In [ ]:
!pip install torch torchvision torchaudio

from torchvision import models, transforms
from PIL import Image
import torch, joblib, numpy as np
import torch.nn as nn

# Load AlexNet and extract feature
model = models.alexnet(pretrained=True)
feature_model = nn.Sequential(
    model.features,
    nn.AdaptiveAvgPool2d((6,6)),
    nn.Flatten(),
    model.classifier[0], model.classifier[1], model.classifier[2],
    model.classifier[3], model.classifier[4], model.classifier[5]
).eval()

tfm = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

img = Image.open("sample.jpg").convert('RGB')
x = tfm(img).unsqueeze(0)
feat = feature_model(x).detach().numpy()

svm = joblib.load("../models/ecoc_svm.pkl")['ecoc']
class_names = joblib.load("../models/ecoc_svm.pkl")['class_names']
pred = svm.predict(feat)[0]
print("Predicted:", class_names[int(pred)])
